In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

Conv2D_Filters = 64
Conv2D_KernelSize = (3, 3)
Conv2D_Activation = "relu"

MaxPool_Size = (2, 2)

Dense1_Units = 512
Dense1_Activation = "relu"

Dense2_Units = 10
Dense2_Activation = "softmax"

LearningRate = 0.01
Decay = 0.00001
Momentum = 0.9

Epochs = 28
BatchSize = 256
ValidationSplit = 0.2

params = [f"Conv2D: {Conv2D_Filters}, {Conv2D_KernelSize}, {Conv2D_Activation}",
          f"MaxPooling2D: {MaxPool_Size}",
          f"Flatten - No Params",
          f"Dense1: {Dense1_Units}, {Dense1_Activation}",
          f"Dense2: {Dense2_Units}, {Dense2_Activation}",
          f"Learning Rate: {LearningRate}",
          f"Decay: {Decay}",
          f"Momentum: {Momentum}",
          f"Epochs: {Epochs}",
          f"Batch Size: {BatchSize}",
          f"Validation Split: {ValidationSplit}"]

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

#x_train = np.reshape(x_train,(50000,32*32*3))
#x_test = np.reshape(x_test,(10000,32*32*3))

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


model = Sequential()
model.add(InputLayer(input_shape=(32,32,3)))
model.add(Conv2D(Conv2D_Filters, Conv2D_KernelSize, activation=Conv2D_Activation))
model.add(MaxPooling2D(pool_size=MaxPool_Size))
model.add(Flatten())
model.add(Dense(Dense1_Units, activation=Dense1_Activation))
model.add(Dense(Dense2_Units, activation=Dense2_Activation))
mySGD = SGD(learning_rate=LearningRate, decay=Decay, momentum=Momentum)

model.compile(optimizer=mySGD, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(x_train, y_train, epochs=Epochs, batch_size=BatchSize, validation_split=ValidationSplit)

training_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)

print(training_score)
print(test_score)

currentTime = datetime.now()
timeStr = currentTime.strftime("%m-%d-%Y_%H:%M:%S")

logFile = open(f"Model2_History_{timeStr}.txt", "w+")

logFile.write("Parameters:\n")
for s in params:
    logFile.write(f"{s}\n")
logFile.write("=================================================\n")

for e in range(Epochs):
    logFile.write(f"Epoch: {e+1}/{Epochs} - ")
    for key in history.history.keys():
        logFile.write(f"{key}: {round(history.history[key][e], 4)} - ")
    logFile.write("\n")
    
logFile.write("=================================================\n")
logFile.write("[Loss, Accuracy]\n")
logFile.write(f"Training Score = {training_score}\n")
logFile.write(f"Test Score = {test_score}\n")
logFile.close()
model.save(f"Model2_{timeStr}.h5")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               7373312   
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 7,380,234
Trainable params: 7,380,234
Non-trainable params: 0
_________________________________________________________________
Epoch 1/28
157/157 [==============================] - 22s 136ms/step - loss: 1.9217 - accuracy: 0.3172 - val_l

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from keras.utils.np_utils import to_categorical


#Change X_Test and Y_Test for a different data set.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_test = to_categorical(y_test, num_classes=10)
x_test = np.reshape(x_test,(10000,32*32*3))
x_test = x_test.astype('float32') / 255.0

testModel = load_model('FINAL_MODEL2.h5')
predictions = testModel.predict(x_test)
predictionClass = []
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

for i in range(len(predictions)):
    imagePrediction = np.where(predictions[i] == predictions[i].max())
    predictionClass.append(classes[int(imagePrediction[0])])

print(predictionClass)